In [27]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [28]:
df = pd.read_csv('Resale flat prices based on registration date from Jan-2017 onwards.csv')

In [29]:
# Convert 'remaining_lease' to a numerical variable (total months)
def lease_to_months(lease):
    parts = lease.split(' ')
    years = int(parts[0])  # Extract years
    months = int(parts[2]) if len(parts) > 2 else 0  # Extract months if present
    return years * 12 + months

df['remaining_lease_months'] = df['remaining_lease'].apply(lease_to_months)

# Extract numerical features from 'storey_range'
def storey_to_avg(storey_range):
    low, high = map(int, storey_range.split(' TO '))
    return (low + high) / 2

df['average_storey'] = df['storey_range'].apply(storey_to_avg)

# Extract 'year' and 'month' from the 'month' column
df['year'] = pd.to_datetime(df['month']).dt.year
df['month_num'] = pd.to_datetime(df['month']).dt.month

# Convert 'block' column from object to int
# df['block'] = df['block'].apply(lambda x: int(x))

# Drop redundant columns
df.drop(['month', 'storey_range', 'remaining_lease'], axis=1, inplace=True)

In [30]:
# One-hot encoding for categorical variables
categorical_cols = ['town', 'flat_type', 'flat_model']
ohe = OneHotEncoder(sparse_output=False)
encoded_features = ohe.fit_transform(df[categorical_cols])
encoded_feature_names = ohe.get_feature_names_out(categorical_cols)
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names)

# Save the encoder for later use
joblib.dump(ohe, 'models/onehotencoder.pkl')

# Concatenate one-hot encoded features with the original DataFrame
df = pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)

In [ ]:
# Scaling numerical columns
numerical_cols = ['floor_area_sqm', 'lease_commence_date', 'remaining_lease_months', 'average_storey', 'year', 'month_num']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[numerical_cols])
scaled_df = pd.DataFrame(scaled_features, columns=numerical_cols)

# Save the encoder for later use
joblib.dump(scaler, 'models/scaler.pkl')

# Concatenate scaled features with the rest of the DataFrame
df = pd.concat([df.drop(columns=numerical_cols), scaled_df], axis=1)

# Final prepared DataFrame
print(df.head())

# Save the prepared dataset to a new file
df.to_csv('preprocessing/hdb_resale_prepared.csv', index=False)

  block        street_name  resale_price  town_ANG MO KIO  town_BEDOK  \
0   406  ANG MO KIO AVE 10      232000.0              1.0         0.0   
1   108   ANG MO KIO AVE 4      250000.0              1.0         0.0   
2   602   ANG MO KIO AVE 5      262000.0              1.0         0.0   
3   465  ANG MO KIO AVE 10      265000.0              1.0         0.0   
4   601   ANG MO KIO AVE 5      265000.0              1.0         0.0   

   town_BISHAN  town_BUKIT BATOK  town_BUKIT MERAH  town_BUKIT PANJANG  \
0          0.0               0.0               0.0                 0.0   
1          0.0               0.0               0.0                 0.0   
2          0.0               0.0               0.0                 0.0   
3          0.0               0.0               0.0                 0.0   
4          0.0               0.0               0.0                 0.0   

   town_BUKIT TIMAH  ...  flat_model_Standard  flat_model_Terrace  \
0               0.0  ...                  0.0  